In [1]:
!pip3 install pandas
!pip3 install folium
!pip3 install dask
!pip3 install "dask[distributed]" --upgrade
!pip3 install aiohttp

^C


In [2]:
import pandas as pd
import folium
import requests
import os
import dask
import glob
from dask.distributed import Client
import dask.dataframe as dd

In [3]:
df_station = pd.read_csv("./isd-history.csv")
df_station
open_station = df_station[df_station["END"] == 20220502]
len(open_station)

353

In [4]:
client = Client(n_workers=4)

In [ ]:
if not os.path.exists("./data"):
    os.mkdir("./data")

for year in range(2007, 2008):
    year = str(year)
    path = "./data/" + year

    if not os.path.exists(path):
        os.mkdir(path)

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if not os.path.exists(path + "/" + station + ".csv"):
            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

            if (requests.get(url).status_code == 200):
                #df = pd.read_csv(url)
                ddf = dd.read_csv(url, dtype='object')
                ddf.to_csv(path + "/" + station + ".csv", index=True, single_file = True)
                #df = pd.read_csv(url).to_csv(path + "/" + station + ".csv", index=True)
                #sleep(1)

KeyboardInterrupt: 

In [5]:
dfs = []

for year in range(2005, 2023):
    year = str(year)
    path = "./data/" + year

    for filename in glob.glob(path + "/*.csv"):
        dfs.append(pd.read_csv(filename))

big_frame = pd.concat(dfs)
big_frame

c:\Python\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,23,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Python\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Python\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19,27,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Python\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Python\lib\site-packages\IPython\core\int

,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,KG1,KG2,MF1,RH1,RH2,RH3,GE1,KA3,KA4,WA1
0,0,72011354829,2005-01-01T00:00:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,72011354829,2005-01-01T00:14:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,72011354829,2005-01-01T00:36:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,72011354829,2005-01-01T00:54:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,72011354829,2005-01-01T01:14:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,4064,72437503893,2022-05-02T03:53:00,6,39.14317,-86.61676,253.3,"BLOOMINGTON MONROE CO AIRPORT, IN US",FM-15,KBMG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4065,4065,72437503893,2022-05-02T04:53:00,6,39.14317,-86.61676,253.3,"BLOOMINGTON MONROE CO AIRPORT, IN US",FM-15,KBMG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4066,4066,72437503893,2022-05-02T04:59:00,6,39.14317,-86.61676,253.3,"BLOOMINGTON MONROE CO AIRPORT, IN US",SOD,KBMG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4067,4067,72437503893,2022-05-02T05:53:00,6,39.14317,-86.61676,253.3,"BLOOMINGTON MONROE CO AIRPORT, IN US",FM-15,KBMG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clean dataframe

In [6]:
big_frame['TMP_control'] = big_frame['TMP'].str.split(pat=',', expand=True)[1]
big_frame['TMP_test'] = big_frame['TMP'].str.split(pat=',', expand=True)[0]
big_frame['TMP_sign'] = big_frame['TMP'].astype(str).str[0]
big_frame['TMP_final'] = (big_frame['TMP_test'].astype(str).str[1:4] + "." + big_frame['TMP_test'].astype(str).str[4:]).astype(float)
big_frame['DATE'] = pd.to_datetime(big_frame['DATE'], format="%Y-%m-%dT%H:%M:%S")
big_frame = big_frame[big_frame['TMP_control'] == "1"]

In [7]:
big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']
big_frame

c:\Python\lib\site-packages\pandas\core\series.py:4576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-7-cfcacddebb15>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']


,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,RH2,RH3,GE1,KA3,KA4,WA1,TMP_control,TMP_test,TMP_sign,TMP_final
0,0,72011354829,2005-01-01 00:00:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0130,1,13.0
1,1,72011354829,2005-01-01 00:14:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0120,1,12.0
2,2,72011354829,2005-01-01 00:36:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0120,1,12.0
3,3,72011354829,2005-01-01 00:54:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0120,1,12.0
4,4,72011354829,2005-01-01 01:14:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",AUTO,K7D2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0120,1,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3712,3712,72436553896,2022-05-01 01:05:00,4,38.24889,-86.95361,161.2,"HUNTINGBURG AIRPORT, IN US",FM-16,99999,...,NaN,NaN,"9,AGL ,+99999,+99999",NaN,NaN,NaN,1,+0190,1,19.0
3713,3713,72436553896,2022-05-01 01:56:00,4,38.24889,-86.95361,161.2,"HUNTINGBURG AIRPORT, IN US",FM-15,99999,...,NaN,NaN,"9,AGL ,+99999,+99999",NaN,NaN,NaN,1,+0183,1,18.3
3714,3714,72436553896,2022-05-01 02:56:00,4,38.24889,-86.95361,161.2,"HUNTINGBURG AIRPORT, IN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0189,1,18.9
3715,3715,72436553896,2022-05-01 03:56:00,4,38.24889,-86.95361,161.2,"HUNTINGBURG AIRPORT, IN US",FM-15,99999,...,NaN,NaN,"9,AGL ,+99999,+99999",NaN,NaN,NaN,1,+0206,1,20.6


In [8]:
cleaned_frame = big_frame[["NAME", "TMP_final", "LATITUDE", "LONGITUDE", "DATE"]]
cleaned_frame

,NAME,TMP_final,LATITUDE,LONGITUDE,DATE
0,"OAKLAND TROY AIRPORT, MI US",13.0,42.54306,-83.17806,2005-01-01 00:00:00
1,"OAKLAND TROY AIRPORT, MI US",12.0,42.54306,-83.17806,2005-01-01 00:14:00
2,"OAKLAND TROY AIRPORT, MI US",12.0,42.54306,-83.17806,2005-01-01 00:36:00
3,"OAKLAND TROY AIRPORT, MI US",12.0,42.54306,-83.17806,2005-01-01 00:54:00
4,"OAKLAND TROY AIRPORT, MI US",12.0,42.54306,-83.17806,2005-01-01 01:14:00
...,...,...,...,...,...
3712,"HUNTINGBURG AIRPORT, IN US",19.0,38.24889,-86.95361,2022-05-01 01:05:00
3713,"HUNTINGBURG AIRPORT, IN US",18.3,38.24889,-86.95361,2022-05-01 01:56:00
3714,"HUNTINGBURG AIRPORT, IN US",18.9,38.24889,-86.95361,2022-05-01 02:56:00
3715,"HUNTINGBURG AIRPORT, IN US",20.6,38.24889,-86.95361,2022-05-01 03:56:00


# Statistiques globales de température (moy, min, max)

In [9]:
temp = cleaned_frame.groupby(by="NAME").agg({'TMP_final': ['mean', 'min', 'max'], "LATITUDE": "first", "LONGITUDE": "first"})
temp

TMP_final              \
                                                         mean   min   max   
NAME                                                                        
ABERDEEN REGIONAL AIRPORT, SD US                     7.355967 -36.7  33.9   
AHOSKIE TRI CO AIRPORT, NC US                       11.822093   0.0  20.2   
ALAMINOS CANYON, TX US                              21.320388  15.0  26.0   
ALEXANDRIA CHANDLER FIELD, MN US                    10.333333   7.0  14.0   
ANDALUSIA OPP AIRPORT, AL US                        16.525000  13.9  21.1   
...                                                       ...   ...   ...   
WAUSAU ASOS, WI US                                   8.555556  -8.0  21.0   
WHARTON REGIONAL AIRPORT, TX US                     18.773529   4.0  28.7   
WILLMAR MUNICIPAL AIRPORT JOHN L RICE FIELD, MN US   8.830972 -25.0  34.0   
WINSTON SALEM REYNOLDS AIRPORT, NC US               14.140000   2.0  28.0   
ZEPHYRHILLS MUNICIPAL AIRPORT, FL US                20.015294   8.9  30.7   

                                                    LATITUDE LONGITUDE  
                                                       first     first  
NAME                                                                    
ABERDEEN REGIONAL AIRPORT, SD US                    45.44330 -98.41300  
AHOSKIE TRI CO AIRPORT, NC US                       36.29750 -77.17083  
ALAMINOS CANYON, TX US                              26.93333 -94.68333  
ALEXANDRIA CHANDLER FIELD, MN US                    45.86790 -95.39410  
ANDALUSIA OPP AIRPORT, AL US                        31.30614 -86.39018  
...                                                      ...       ...  
WAUSAU ASOS, WI US                                  44.92880 -89.62770  
WHARTON REGIONAL AIRPORT, TX US                     29.26611 -96.00750  
WILLMAR MUNICIPAL AIRPORT JOHN L RICE FIELD, MN US  45.11700 -95.13300  
WINSTON SALEM REYNOLDS AIRPORT, NC US               36.13361 -80.22222  
ZEPHYRHILLS MUNICIPAL AIRPORT, FL US                28.22813 -82.15592  

[201 rows x 5 columns]

# Affichage Station (MAP)

In [10]:
m = folium.Map(location=[temp["LATITUDE"].mean(), temp["LONGITUDE"].mean()], zoom_start=5)

for i in range(0,len(temp)):
   folium.Marker(
      location=[temp.iloc[i]['LATITUDE'], temp.iloc[i]['LONGITUDE']],
      popup=folium.Popup(temp.iloc[i].to_frame().to_html(), max_width=450,min_height=400)
   ).add_to(m)
   
m

In [ ]:
#dtypes = {
#'STATION':         int,
#'DATE':           object,
#'SOURCE':         object,
#'LATITUDE':      float,
#'LONGITUDE':     float,
#'ELEVATION':     float,
#'NAME':           object,
#'REPORT_TYPE':    object,
#'CALL_SIGN':      object,
#'QUALITY_CONTROL':object,
#'WND':            object,
#'CIG':            object,
#'VIS':            object,
#'TMP':            object,
#'DEW':            object,
#'SLP':            object,
#'AA1':            object,
#'AT1':            object,
#'AT2':            object,
#'AT3':            object,
#'AT4':            object,
#'AT5':            object,
#'AU1':            object,
#'AW1':            object,
#'OC1':            object,
#'REM':            object,
#'EQD':            object
#}
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year
#
#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);
#
#        while (len(wban) < 5):
#            wban = '0' + wban
#
#        station  = str(station_row.USAF) + wban
#
#        if os.path.exists(path + "/" + station + ".csv"):
#            #df = pd.read_csv(path + "/" + station + ".csv")
#            #print(df.dtypes)
#            df = dd.read_csv(path + "/" + station + ".csv", dtype=dtypes)
#            
#            
#            #df = dd.read_csv(path + "/" + station + ".csv")
#            #df.head()


In [ ]:
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year

#    if not os.path.exists(path):
#        os.mkdir(path)

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban

#        if not os.path.exists(path + "/" + station + ".csv"):
#            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

#            if (requests.get(url).status_code == 200):
#                pd.read_csv(url).to_csv(path + "/" + station + ".csv")

#            spark.sparkContext.addFile('https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv')

#            if (SparkFiles.get(station + ".csv")):
#                df = spark.read.csv("file://"+SparkFiles.get(station + ".csv"), header=True)

#                print("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")
#                df.write.format("csv").save("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")

#for year in range(2022, 2023):

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban
        
#        df = spark.read.csv("hdfs://namenode:9000/noaa/" + str(year) + "/" + station + ".csv", header=True)
#df = spark.read.csv("hdfs://namenode:9000/noaa/2022/72026853882.csv", header=True)
#df.take(5)